In [8]:
!pip -qq install torchtext==0.3.1
!pip -qq install autocorrect

     |████████████████████████████████| 622 kB 5.0 MB/s 


In [4]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

np.random.seed(313)

In [5]:
import math
from tqdm import tqdm
tqdm.get_lock().locks = []

In [6]:
torch.torch_version.internal_version

'1.12.1+cu113'

In [7]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import urllib.parse

## Загрузка данных

In [12]:
CONNECTION_STRING = ""

In [20]:
import re


def atoi(text):
    return int(text) if text.isdigit() else text


def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


def get_latest_preprocessed_depression_df(do_remove_id=True, do_remove_age=True):
    client = MongoClient(CONNECTION_STRING)
    print(client.list_database_names())
    db = client['depression']
    print(db.list_collection_names())
    preprocessed_collections = [coll for coll in db.list_collection_names() if 'preprocessed_depression_data' in coll]
    preprocessed_collections.sort(key=natural_keys)
    latest_collection_name = preprocessed_collections[-1]
    latest_preprocessed_data_collection = db[latest_collection_name]
    preprocessed_data_cursor = latest_preprocessed_data_collection.find()
    dataframe = pd.DataFrame(list(preprocessed_data_cursor))
    if do_remove_id:
        del dataframe['_id']
    if do_remove_age:
        del dataframe['age']
    print(dataframe)
    return dataframe

depression_df = get_latest_preprocessed_depression_df()

['depression']
['raw_depression_data', 'preprocessed_depression_data_19_11_2022_13_24_38', 'depression_data_accessors']
                                                    text  label
0      добрый романтик который стремиться помогать по...      1
1      проблема девушка каждый ссора мочь перенести о...      1
2      весь жизнь это сплошной ад который горе постоя...      1
3      хотеть уснуть просыпаться каждый день время сч...      1
4      здраствовать каждый день просыпаться мысль жит...      1
...                                                  ...    ...
62553  минувший выходной спортблок калмга проходить з...      0
62554  v сезон club129270552 международный детский не...      0
62555  втб подводить итог первый корпоративный акселе...      0
62556  втб запускать бесплатный пополнение карта банк...      0
62557  сызрань вновь пройти ярмарка вакансия рабочий ...      0

[62558 rows x 2 columns]


##Разделение на train, val, test

In [21]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(depression_df, test_size=0.20, random_state=313, stratify=depression_df['label'])
train_df, val_df =  train_test_split(train_df, test_size=0.10, random_state=313, stratify=train_df['label'])

print(train_df)
print(val_df)
print(test_df)

                                                    text  label
50956  полицейский подводить промежуточный итог акция...      0
16176  привет артем военнослужащий 27 потерять жена с...      1
23017  бросать институт мама узнавать переживать люби...      1
33949  центр столица определяться застройщик участок ...      0
22248  здравствовать 26 звать саша писать получать от...      1
...                                                  ...    ...
18392  здраствовать звать оксана 14лет 8 год жить опе...      1
12909  подсказывать находить сила сломаться обстоятел...      1
19531  привет казаться жизнь никто держать семья един...      1
58635  работник управляющий компания сибирский альянс...      0
11559  человек здравствовать 14 дет хотеть умирать по...      1

[45041 rows x 2 columns]
                                                    text  label
54419  хотеть посоветовать проверенный лицензионный к...      0
31791  дом культура сердце садиться название нерица п...      0
39119  летний 

In [22]:
def df_to_list(df):
  data_list = []
  for idx, data_row in df.iterrows():
    data_list.append(([word.strip() for word in data_row['text'].strip().split()], data_row['label']))
  print(data_list[0])
  return data_list


In [23]:
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

tokens_field = Field()
label_field = LabelField()

fields = [('tokens', tokens_field), ('label', label_field)]

train_dataset = Dataset([Example.fromlist(example, fields) for example in df_to_list(train_df)], fields)
val_dataset = Dataset([Example.fromlist(example, fields) for example in df_to_list(val_df)], fields)
test_dataset = Dataset([Example.fromlist(example, fields) for example in df_to_list(test_df)], fields)

tokens_field.build_vocab(train_dataset)
label_field.build_vocab(train_dataset)

print('Vocab size =', len(tokens_field.vocab))

train_iter, val_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset, test_dataset), batch_sizes=(32, 128, 128), 
    shuffle=True, device=DEVICE, sort=False
)

print (train_dataset)

(['полицейский', 'подводить', 'промежуточный', 'итог', 'акция', 'выкуп', 'незаконный', 'оружие', 'население', 'тысяча', 'единица', 'огнестрельный', 'оружие', 'сдавать', 'полиция', 'казахстанец'], 0)
(['хотеть', 'посоветовать', 'проверенный', 'лицензионный', 'казино', 'чемпион', 'новый', 'игрок', 'бонус', '200', '000', 'рубль', 'первый', '5', 'депозит', 'выплата', 'течение', 'час', 'любой', 'сумма', 'любой', 'платежный', 'система', 'первый', 'выплата', 'верификация', 'интересно', 'ссылка', 'чемпион', 'игровой', 'автомат', 'онлайн', 'официальный', 'сайт', 'champion', 'casino', 'играть', 'бесплатно', 'рег'], 0)
(['сегодня', 'club109344729', 'волонт', 'ры', 'победа', 'проводить', 'субботник', 'ботанический', 'сад', '100', 'активист', 'движение', 'школьник', 'студент', 'надевать', 'перчатка', 'очищать', 'мусор', 'масштабный', 'территория', 'сад', 'волонтер', 'победа', 'организовывать', 'раздельный', 'сбор', 'мусор', 'пластик', 'стекло', 'отходы', 'рассортировывать', 'разный', 'пакет', 'отпр

In [24]:
for ex in train_iter.dataset.examples:
  print(str(ex.tokens) + "   " + str(ex.label))

Output hidden; open in https://colab.research.google.com to view.

## Классификатор депрессии

In [13]:
class DepressionClassifierModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=64, lstm_hidden_dim=128, num_layers=1, dropout_p=0.2):
        super().__init__()

        self.embeddings_layer = nn.Embedding(vocab_size, emb_dim)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm_layer = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, bidirectional=True, num_layers=num_layers, batch_first=True)
        self.out_layer = nn.Linear(lstm_hidden_dim * 2, 2)


    def forward(self, inputs):
        projections = self.embeddings_layer(inputs)
        _, (final_hidden_state, _) = self.lstm_layer(projections)
        hidden = self.dropout(torch.cat([final_hidden_state[0], final_hidden_state[1]], dim=1))
        output = self.out_layer(hidden)
        return output

In [14]:
class ModelTrainer():
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self.epoch_loss = 0
        self.correct_count, self.total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        
        self.model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        logits = self.model(batch.tokens.transpose(0, 1))

        loss = self.criterion(logits, batch.label)

        predicted_label = logits.argmax(dim=1)

        self.total_count += predicted_label.size(0)
        self.correct_count += torch.sum(predicted_label == batch.label).item()

        if self.is_train:
          loss.backward()
          self.optimizer.step()
          self.optimizer.zero_grad()
        self.epoch_loss += loss.item()

In [15]:
def do_epoch(trainer, data_iter, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=trainer.batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()

            
def fit(trainer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, is_train=True, name=name_prefix + 'Train:')
        
        if not val_iter is None:
            do_epoch(trainer, val_iter, is_train=False, name=name_prefix + '  Val:')

In [14]:
model = DepressionClassifierModel(vocab_size=len(tokens_field.vocab)).to(DEVICE)

criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())

trainer = ModelTrainer(model, criterion, optimizer)

fit(trainer, train_iter, epochs_count=20, val_iter=val_iter)

[1 / 30] Train: Loss = 0.11851, Accuracy = 95.68%: 100%|██████████| 1408/1408 [00:41<00:00, 33.63it/s]
[1 / 30]   Val: Loss = 0.06032, Accuracy = 97.70%: 100%|██████████| 40/40 [00:02<00:00, 14.18it/s]
[2 / 30] Train: Loss = 0.05379, Accuracy = 98.32%: 100%|██████████| 1408/1408 [00:41<00:00, 33.71it/s]
[2 / 30]   Val: Loss = 0.04996, Accuracy = 98.84%: 100%|██████████| 40/40 [00:02<00:00, 14.07it/s]
[3 / 30] Train: Loss = 0.02108, Accuracy = 99.31%: 100%|██████████| 1408/1408 [00:41<00:00, 33.65it/s]
[3 / 30]   Val: Loss = 0.03579, Accuracy = 98.96%: 100%|██████████| 40/40 [00:02<00:00, 14.88it/s]
[4 / 30] Train: Loss = 0.01147, Accuracy = 99.65%: 100%|██████████| 1408/1408 [00:41<00:00, 33.76it/s]
[4 / 30]   Val: Loss = 0.03548, Accuracy = 98.98%: 100%|██████████| 40/40 [00:02<00:00, 15.15it/s]
[5 / 30] Train: Loss = 0.00665, Accuracy = 99.84%: 100%|██████████| 1408/1408 [00:41<00:00, 33.72it/s]
[5 / 30]   Val: Loss = 0.03684, Accuracy = 99.06%: 100%|██████████| 40/40 [00:02<00:00, 1

**Задание** Подсчитайте итоговое качество на тесте.

In [15]:
do_epoch(trainer, test_iter, is_train=False, name="Test:")

Test: Loss = 0.10024, Accuracy = 99.13%: 100%|██████████| 98/98 [00:06<00:00, 15.83it/s]


In [19]:
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/doom-model-demo')

In [17]:
loaded_model = torch.load('/content/drive/MyDrive/Colab Notebooks/doom-model-demo')

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Тестирование на произвольных запросах**

---



In [18]:
from autocorrect import Speller
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem

nltk.download("stopwords")
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

speller = Speller('ru')

def preprocess_text(text):
  # remove links
  text = ' '.join(re.sub("http\S+", " ", text).split())
  # remove hashtags, @mentions, emojis
  text = ' '.join(re.sub("(@[А-Яа-яA-Za-z0-9]+)|(#[А-Яа-яA-Za-z0-9]+)|(<Emoji:.*>)", " ", text).split())
  # remove punctuation and fix word-dot-word without space
  text = ' '.join(re.sub("([^0-9A-Za-zА-Яа-я \t])", " ", text).split())
  # try to fix typos

  text = speller(text)
  tokens = mystem.lemmatize(text.lower())
  tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
  text = " ".join(tokens)
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


dictionary for this language not found, downloading...
__________________________________________________
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
done!


In [29]:
def infer(text, model = model):
  print('---------')
  print('Sentence: ' + text)
  print('---------\n')
  tokens = torch.IntTensor([[tokens_field.vocab.stoi[i] for i in text.split()]]).to(DEVICE)
  predicted_label = model(tokens).argmax(dim=1)
  print(predicted_label)


infer('как же хочется сдохнуть я просто не могу так жить аааа')
infer('Играй в war thunder по ссылке в описании, вертолеты, танки, самолеты, тут все есть че тебе надо')
infer('Я обожаю свою жизнь, как же хорошо жить на этом свете')
infer('суицид - штука хорошая')
infer('абсллютно случайный текст, реагирую на новость, которую увидел в каком-то паблике, должна быть такая же маршрутка с охапками роз, согласны, узнали?')
infer('как же тяжело и больно')
infer('здравствуйте мне 12 лет')
infer('здравствуйте')
infer('мне 12 лет')


---------
Sentence: как же хочется сдохнуть я просто не могу так жить аааа
---------

tensor([1])
---------
Sentence: Играй в war thunder по ссылке в описании, вертолеты, танки, самолеты, тут все есть че тебе надо
---------

tensor([0])
---------
Sentence: Я обожаю свою жизнь, как же хорошо жить на этом свете
---------

tensor([0])
---------
Sentence: суицид - штука хорошая
---------

tensor([1])
---------
Sentence: абсллютно случайный текст, реагирую на новость, которую увидел в каком-то паблике, должна быть такая же маршрутка с охапками роз, согласны, узнали?
---------

tensor([0])
---------
Sentence: как же тяжело и больно
---------

tensor([1])
---------
Sentence: здравствуйте мне 12 лет
---------

tensor([0])
---------
Sentence: здравствуйте
---------

tensor([0])
---------
Sentence: мне 12 лет
---------

tensor([1])
